In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./데이터/data.csv')

# 채점 모델 만들기

## 데이터셋 분리

In [9]:
from sklearn.model_selection import train_test_split

# train, test 데이터셋 분리
X = data[data.columns[:-1]]
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y)

## 모델 생성

In [20]:
from sklearn.metrics import accuracy_score
import joblib

### 1. SVC

In [10]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# # 정규화 작업
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# SVM 모델 생성
sv_clf = SVC(kernel='poly', C = 3, degree = 3, probability=True)
sv_clf.fit(X_train, y_train)

# # test 데이터셋도 정규화(train 데이터셋 기준으로 학습시킨 정규화 모듈 사용)
# X_test = scaler.transform(X_test)

svm_pred = sv_clf.predict(X_test) # 예측 라벨
accuracy_score(y_test, svm_pred)

0.7315634218289085

### 2. RandomForest

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics    
 
# 학습 진행
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train, y_train)

# 예측
rf_pred = forest.predict(X_test)

# 정확도 확인
print('정확도 :', metrics.accuracy_score(y_test, rf_pred))

정확도 : 0.7876106194690266


### 3. 로지스틱 회귀

In [12]:
from sklearn.linear_model import LogisticRegression

logit_clf = LogisticRegression()
logit_clf.fit(X_train, y_train)

lr_pred = logit_clf.predict(X_test)
accuracy_score(y_test, lr_pred)

0.7551622418879056

### 4. XGBoost

In [13]:
import xgboost as xgb

# XGBoost 모델 초기화 및 학습
model = xgb.XGBClassifier(objective="multi:softmax", num_class=3)  # 다중 클래스 분류를 위한 설정
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7581120943952803


In [23]:
joblib.dump(sv_clf, 'SVC.pkl')
joblib.dump(forest, 'RForest.pkl')
joblib.dump(logit_clf, 'Logistic.pkl')
joblib.dump(model, 'XGBoost.pkl')

['XGBoost.pkl']